In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
#Clear the previous tensorflow graph
K.clear_session()

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(50, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_3(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(100, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_4(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [8]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the results of running DE on the old model (250->50->1)
windowSize = 20
stride = 2
constRUL = 105

dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)

In [9]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
#windowSizes = {1:25,2:15,3:25,4:15}
windowSizes = 15
stride = 2
#constRUL = {1:93, 2:94, 3:92, 4:94}
constRUL = 95
#models = {1:RULmodel_SN_1,2:RULmodel_SN_2,3:RULmodel_SN_3,4:RULmodel_SN_4}
models = {1:RULmodel_SN_1}

file = open("ResultsAllDatasets3_1.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

for key, model in models.items():
    
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
    
    for i in range(1,5):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')

        #Create and compile the models
        windowSize = windowSizes
        nFeatures = len(selected_features)
        shapeSN = nFeatures*windowSize
        modelRULSN = model(shapeSN)
        modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

        tModel.changeModel('ModelRUL_SN_Dat'+str(shapeSN), modelRULSN)
        tModel.windowSize = windowSizes
        tModel.windowStride = stride
        tModel.constRul = constRUL
        tModel.changeDataset(dataset)
        tModel.loadData()
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))

        for j in range(iterations):

            print("Iteration "+str(j+1))
            #file.write("Iteration "+str(j+1)+'\n\n')
            tModel.trainModel(learningRateScheduler=lrate, verbose=0)
            tModel.evaluateModel(["rhs"], round=True)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.trainTime

            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        #print(tempScoresRMSE)
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        #file.write(str(stats.describe(tempScoresRMSE))+'\n\n')
        #file.write(str(stats.describe(tempScoresRHS))+'\n\n')
        #file.write(str(stats.describe(tempTime))+'\n\n')
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
        
file.close()

For model 1
Computing for dataset 1
Iteration 1


KeyboardInterrupt: 

In [3]:
#Clear the previous tensorflow graph
K.clear_session()

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

dataFolder = '../CMAPSSData'

datasets = [1]
iterations = 10
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
#windowSizes = {1:25,2:15,3:25,4:15}
windowSizes = 15
stride = 2
#constRUL = {1:93, 2:94, 3:92, 4:94}
constRUL = 95
#models = {1:RULmodel_SN_1,2:RULmodel_SN_2,3:RULmodel_SN_3,4:RULmodel_SN_4}
models = {1:RULmodel_SN_1}

#Create and compile the model
windowSize = 15
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData()
tModel.epochs = 200

#file = open("ResultsBestModelAllDatasetsSingleSet.csv", "w")
#csvfile = csv.writer(file, lineterminator='\n')

min_rmse = 1000
bestModel = None
bestIndex = 0

for i in range(iterations):

    print("Iteration "+str(i+1))
    #file.write("Iteration "+str(j+1)+'\n\n')
    tModel.trainModel(learningRateScheduler=lrate, verbose=0)
    tModel.evaluateModel(["rhs"], round=True)

    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rmse2 = cScores['rmse']
    rhs = cScores['rhs']
    time = tModel.trainTime

    if rmse2 < min_rmse:
        bestModel = tModel.model
        bestIndex = i
    
    tempScoresRMSE[i] = rmse2
    tempScoresRHS[i] = rhs
    tempTime[i] = time

print(tempScoresRMSE)    
    
#save best model to file
# serialize model to JSON
model_json = bestModel.to_json()
with open("bestRULModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
bestModel.save_weights("bestRULModel.h5")
print("Saved model to disk")
        
#file.close()




















Iteration 1
100/100 [==============================] - 0s 291us/step


NameError: name 'tempScoresRMSE' is not defined